# Logisitc Regession

In [ ]:
# Import Dependencies
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import Data
df = pd.read_csv('data.csv')  

In [ ]:
# Create X and Y
X = df.drop(['y_var'], axis=1)
y = df['y_var']

In [ ]:
# Use the following to encode cateogorical dependent variables:
from sklearn.preprocessing import OneHotEncoder
oenc=OneHotEncoder(drop='first')
multiple_enc=oenc.fit_transform(X[['Venue','Day','Time','Captain','Opposition']])# onehotencoding on  features
multiple_enc=multiple_enc.toarray()

#oenc.get_feature_names() use this method to get column names
multiple_enc=pd.DataFrame(multiple_enc,columns=oenc.get_feature_names_out())
multiple_enc.head()
X = pd.concat([X[["PrevFandA","WinPreviousTimePlayingSameTeam","Rank"]],multiple_enc],axis=1) # append to original dataframe
X.head()

In [ ]:
# Create Training and Test Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

In [ ]:

log_reg = sm.Logit(y_train, X_train).fit() # fit model

print(log_reg.summary()) #

In [ ]:
# Predict on test data
y_pred_test = log_reg.predict(exog=X_test)
# Convert predicted probabilities to binary class (0 or 1)
y_pred_test = y_pred_test.apply(lambda x: 1 if x >= 0.5 else 0)


In [ ]:
# Get test accuaracy
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))


In [ ]:
#compare the train-set and test-set accuracy to check for overfitting.
y_pred_train = log_reg.predict(exog=X_train)
# Convert predicted probabilities to binary class (0 or 1)
y_pred_train = y_pred_train.apply(lambda x: 1 if x >= 0.5 else 0)

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

These two accuracies should be close if not overfitting has occured

## Logistic regression test assumptions

When the assumptions of logistic regression analysis are not met, problems such as biased coefficient estimates or very large standard errors for the logistic regression coefficients may lead to invalid statistical inferences.

Assumption 1 - Appropriate outcome type (Must be categorical)

Assumption 2 - Linearity of independent variables and log odds

Assumption 3 - No strongly influential outliers

Assumption 4 - Absence of multicollinearity

Assumption 5 - Independence of observations

Assumption 6 - Sufficiently large sample size

Using: https://github.com/sandipanpaul21/Logistic-regression-in-python/blob/main/07_LR_Assumptions.ipynb

In [ ]:
# Assumption 3:
from scipy import stats

# Get influence measures
influence = log_reg.get_influence()

# Obtain summary df of influence measures
summ_df = influence.summary_frame()

# Filter summary df to Cook distance
diagnosis_df = summ_df.loc[:,['cooks_d']]

# Append absolute standardized residual values
diagnosis_df['std_resid'] = stats.zscore(log_reg.resid_pearson)
diagnosis_df['std_resid'] = diagnosis_df.loc[:,'std_resid'].apply(lambda x: np.abs(x))

# Sort by Cook's Distance
diagnosis_df.sort_values("cooks_d", ascending=False)
diagnosis_df.head()

In [ ]:
# Set Cook's distance threshold
cook_threshold = 4 / len(X)
print(f"Threshold for Cook Distance = {cook_threshold}")

# Plot influence measures (Cook's distance)
fig = influence.plot_index(y_var="cooks", threshold=cook_threshold)
plt.axhline(y = cook_threshold, ls="--", color='red')
fig.tight_layout(pad=2)

In [ ]:
# Find number of observations that exceed Cook's distance threshold
outliers = diagnosis_df[diagnosis_df['cooks_d'] > cook_threshold]
prop_outliers = round(100*(len(outliers) / len(X)),1)
print(f'Proportion of data points that are highly influential = {prop_outliers}%')

In [ ]:

# Find number of observations which are BOTH outlier (std dev > 3) and highly influential
extreme = diagnosis_df[(diagnosis_df['cooks_d'] > cook_threshold) & 
                       (diagnosis_df['std_resid'] > 3)]
prop_extreme = round(100*(len(extreme) / len(X)),1)

In [ ]:

# Display top 5 most influential outliers
extreme.sort_values("cooks_d", ascending=False).head()

- It is important to note that for data points with relative high Cook's distances, it does not automatically mean that it should be immediately removed from the dataset.
- It is essentially an indicator to highlight which data points are worth looking deeper into, to understand whether they are true anomalies or not
- In practice, an assessment of “large” values is a judgement call based on experience and the particular set of data being analyzed.
- In addition, based on our pre-defined threshold (4/N), only 5% (51/891) of the points are in the outlier zone, which is small as well. 
- The issue comes when there is a significant number of data points classified as outliers.

#

In [ ]:
# Assumption 4:
# Use variance inflation factor to identify any significant multi-collinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df):
    vif = pd.DataFrame()
    vif["variables"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return(vif)

calc_vif(X) # If >=5 Severe multicollineairty present

In [ ]:
# Assumption 5: Checking for autocorrelation

from statsmodels.stats.stattools import durbin_watson

#perform Durbin-Watson test
durbin_watson(log_reg.resid)

In [ ]:
#perform Jarque-Bera test to test for normal errors
stats.jarque_bera(log_reg.resid)
# If errors arent normal used residual based bootstrapping

In [ ]:
# Maybe test for heteroskedasticity